In [44]:
import pandas as pd
import numpy as np

In [186]:
df = pd.read_csv('/Users/serdarevichar/Library/CloudStorage/GoogleDrive-serdarevichar@gmail.com/My Drive/fantasy-football-database.csv', index_col = 'Unnamed: 0')

temp1 = df[['Year','Week','Playoff Flag','Home Team','Home Score','Away Score']].copy()
temp2 = df[['Year','Week','Playoff Flag','Away Team','Away Score','Home Score']].copy()

temp1.rename(columns={'Home Team':'Team', 'Home Score':'Score', 'Away Score':'Opp Score'}, inplace=True)
temp2.rename(columns={'Away Team':'Team', 'Away Score':'Score', 'Home Score':'Opp Score'}, inplace=True)

df_converted = pd.concat([temp1,temp2], axis=0, ignore_index=True)
df_converted['Win'] = df_converted['Score'] > df_converted['Opp Score']
df_converted['Win'] = df_converted['Win'].map(int)
df_converted

df_converted.head()

,Year,Week,Playoff Flag,Team,Score,Opp Score,Win
0,2019,1,False,Justin,158.02,113.20,1
1,2019,1,False,Dante,107.92,126.14,0
2,2019,1,False,Michael,112.68,134.32,0
3,2019,1,False,Andrew,128.54,141.26,0
4,2019,1,False,Haris,135.80,101.82,1


# Weekly Scoreboard Table





In [89]:
week = df.loc[(df['Year'] == 2019) & (df['Week'] == 1)]
week

,Year,Week,Playoff Flag,Home Team,Home Score,Home Projection,Away Team,Away Score,Away Projection
0,2019,1,False,Justin,158.02,115.55,The,113.20,121.01
1,2019,1,False,Dante,107.92,118.65,Zach,126.14,114.46
2,2019,1,False,Michael,112.68,118.23,McGwire,134.32,122.20
3,2019,1,False,Andrew,128.54,121.59,Carter,141.26,117.11
4,2019,1,False,Haris,135.80,119.44,Ethan,101.82,112.52
5,2019,1,False,Noah,153.86,125.17,Nathan,132.22,116.77


In [7]:
scoreboard = week[['Home Team','Home Score','Away Score','Away Team']].copy()
scoreboard

,Home Team,Home Score,Away Score,Away Team
0,Justin,158.02,113.20,The
1,Dante,107.92,126.14,Zach
2,Michael,112.68,134.32,McGwire
3,Andrew,128.54,141.26,Carter
4,Haris,135.80,101.82,Ethan
5,Noah,153.86,132.22,Nathan


# Season Standings Table

In [127]:
standings = df.loc[(df['Year'] == 2019) & (df['Week'] < 10)].copy()
standings.head(5)

,Year,Week,Playoff Flag,Home Team,Home Score,Home Projection,Away Team,Away Score,Away Projection
0,2019,1,False,Justin,158.02,115.55,The,113.20,121.01
1,2019,1,False,Dante,107.92,118.65,Zach,126.14,114.46
2,2019,1,False,Michael,112.68,118.23,McGwire,134.32,122.20
3,2019,1,False,Andrew,128.54,121.59,Carter,141.26,117.11
4,2019,1,False,Haris,135.80,119.44,Ethan,101.82,112.52


In [128]:
temp1 = standings[['Home Team','Home Score','Away Score']].copy()
temp2 = standings[['Away Team','Away Score','Home Score']].copy()

temp1.rename(columns={'Home Team':'Team', 'Home Score':'Score', 'Away Score':'Opp Score'}, inplace=True)
temp2.rename(columns={'Away Team':'Team', 'Away Score':'Score', 'Home Score':'Opp Score'}, inplace=True)

standings_data = pd.concat([temp1,temp2], axis=0, ignore_index=True)
standings_data['Win'] = standings_data['Score'] > standings_data['Opp Score']
standings_data['Win'] = standings_data['Win'].map(int)
standings_data

standings_data.head()

,Team,Score,Opp Score,Win
0,Justin,158.02,113.20,1
1,Dante,107.92,126.14,0
2,Michael,112.68,134.32,0
3,Andrew,128.54,141.26,0
4,Haris,135.80,101.82,1


In [122]:
teams = standings_data['Team'].unique()
records = []

for team in teams:
    wins = standings_data.loc[(standings_data['Team'] == team) & (standings_data['Win'] == 1), 'Win'].sum()
    losses = 4 - wins
    points_for = round(standings_data.loc[(standings_data['Team'] == team), 'Score'].sum(),2)
    points_against = round(standings_data.loc[(standings_data['Team'] == team), 'Opp Score'].sum(),2)
    records.append([team, wins, losses, points_for, points_against])

records

[['Justin', 3, 1, 515.7, 432.76],
 ['Dante', 2, 2, 455.06, 449.84],
 ['Michael', 2, 2, 490.94, 568.2],
 ['Andrew', 1, 3, 540.02, 570.16],
 ['Haris', 2, 2, 471.32, 478.12],
 ['Noah ', 2, 2, 534.0, 469.4],
 ['The', 3, 1, 462.34, 435.56],
 ['McGwire', 2, 2, 425.52, 448.94],
 ['Carter', 3, 1, 460.76, 433.56],
 ['Ethan', 1, 3, 410.36, 462.08],
 ['Nathan', 2, 2, 514.72, 514.76],
 ['Zach', 1, 3, 443.16, 460.52]]

In [123]:
table = pd.DataFrame(records, columns=['Team','Wins','Losses', 'Points For','Points Against'])
table.sort_values(['Wins','Points For'], ascending=False)

,Team,Wins,Losses,Points For,Points Against
0,Justin,3,1,515.70,432.76
6,The,3,1,462.34,435.56
8,Carter,3,1,460.76,433.56
5,Noah,2,2,534.00,469.40
10,Nathan,2,2,514.72,514.76
2,Michael,2,2,490.94,568.20
4,Haris,2,2,471.32,478.12
1,Dante,2,2,455.06,449.84
7,McGwire,2,2,425.52,448.94
3,Andrew,1,3,540.02,570.16


# Luck Score Table




In [125]:
standings_data.head()

,Team,Score,Opp Score,Win
0,Justin,158.02,113.20,1
1,Dante,107.92,126.14,0
2,Michael,112.68,134.32,0
3,Andrew,128.54,141.26,0
4,Haris,135.80,101.82,1


In [156]:
mean = standings_data['Score'].mean()
std = standings_data['Score'].std() * 0.5

print(mean,std)

119.09870370370369 11.986959368542161


In [76]:
def opp_luck_score(score, mean=None, std=None):
    return int(((mean - score) / np.absolute(mean - score)) * np.floor(np.absolute(mean - score) / std))

In [157]:
opp_luck_score(100,mean=mean,std=std)

1

In [158]:
standings_data['Opp Luck Score'] = standings_data[['Opp Score']].map(opp_luck_score, mean=mean, std=std)
standings_data

,Team,Score,Opp Score,Win,Opp Luck Score,Your Luck Score
0,Justin,158.02,113.20,1,0,0
1,Dante,107.92,126.14,0,0,0
2,Michael,112.68,134.32,0,-1,0
3,Andrew,128.54,141.26,0,-1,0
4,Haris,135.80,101.82,1,1,0
...,...,...,...,...,...,...
103,Michael,108.56,109.78,0,0,0
104,Dante,117.50,133.12,0,-1,0
105,Justin,115.30,122.36,0,0,0
106,Zach,115.80,100.66,1,1,0


In [139]:
def your_luck_score(score, result, mean=None, std=None):
    xi = lambda x: np.floor((mean + x) / (2*mean)) 
    return int(np.floor(np.absolute(mean - score) / std) * (result * xi(2*mean - score) - (1-result) * xi(score)))

In [141]:
your_luck_score(175,0, mean=mean, std=std)

-2

In [162]:
yls = [your_luck_score(score, result, mean=mean, std=std) for score, result in standings_data[['Score','Win']].values]
standings_data['Your Luck Score'] = yls
standings_data.sort_values('Score', ascending=False).head(10)

,Team,Score,Opp Score,Win,Opp Luck Score,Your Luck Score
30,The,183.40,122.04,1,0,0
70,Noah,181.58,108.72,1,0,0
83,Michael,172.26,172.16,1,-4,0
29,Dante,172.16,172.26,0,-4,-4
28,Justin,164.64,101.92,1,1,0
25,Carter,163.82,114.76,1,0,0
77,Michael,162.32,146.48,1,-2,0
48,Andrew,161.64,137.32,1,-1,0
0,Justin,158.02,113.20,1,0,0
43,Andrew,155.50,108.74,1,0,0


In [164]:
def close_luck_score(score,opp_score):
    if (score - opp_score) < 3 and (score - opp_score) > 0:
        return 1
    elif (score - opp_score) < 0 and (score - opp_score) > -3:
        return -1
    else:
        return 0    

In [171]:
cls = [close_luck_score(score, opp_score) for score, opp_score in standings_data[['Score','Opp Score']].values]
standings_data['Close Luck Score'] = cls
standings_data['Luck Score'] = standings_data[['Opp Luck Score','Your Luck Score','Close Luck Score']].sum(axis=1)


# Champions Table

In [193]:
champions = [(2019, 'Dante'), (2020, 'Ethan'), (2021, 'Andrew'), (2022, 'Dante'), (2023, 'Haris'), (2024, 'Zach')]
champions

df_converted.loc[(df_converted['Week'] == 17) & (df_converted['Win'] == 1), ['Year','Team']].values

array([[2020, 'Ethan'],
       [2021, 'Andrew'],
       [2022, 'Dante'],
       [2023, 'Haris'],
       [2024, 'Zach'],
       [2019, 'Dante']], dtype=object)

In [189]:
champion_dfs = []
for year, team in champions:
    temp = df_converted.loc[(df_converted['Year'] == year) & (df_converted['Team'] == team)]
    champion_dfs.append(temp)

    


champion_data = pd.concat(champion_dfs, axis = 0)
champion_dfs[3].sort_values('Week')

,Year,Week,Playoff Flag,Team,Score,Opp Score,Win
232,2022,1,False,Dante,117.90,64.20,1
736,2022,2,False,Dante,107.36,103.76,1
246,2022,3,False,Dante,141.80,80.70,1
750,2022,4,False,Dante,180.24,134.68,1
756,2022,5,False,Dante,104.08,120.64,0
264,2022,6,False,Dante,101.24,82.50,1
766,2022,7,False,Dante,84.56,123.30,0
274,2022,8,False,Dante,167.54,139.10,1
776,2022,9,False,Dante,125.40,125.84,0
781,2022,10,False,Dante,102.70,117.68,0
